In [1]:
!pip install accelerate -U
!pip install transformers --upgrade
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 16.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [3]:
wine_data = pd.read_csv('/content/drive/MyDrive/datasets/wines_processed/wines_vectors.csv')

In [4]:
wine_data.sample(10)

,style_name,price,currency,volume,acidity,fizziness,sweetness,tannin,user_structure_count,calculated_structure_count,...,region_name,country,winery,image,combined_reviews,flavours,flavours_text,flavours_joined_text,combined_text,combined_text_preprocessed
6259,Moscato,16.99,USD,750,4.467013,0.0,4.990000,3.274588,0.0,93.0,...,Madera,United States,Quady,//images.vivino.com/thumbs/BhKipDz9QOeCF18QNqe...,"SEP Smooth, sweet dessert wine. Excellent with...","['dandelion', 'honeydew melon', 'stone', 'ston...","['dandelion', 'honeydew melon', 'stone', 'ston...","dandelion, honeydew melon, stone, stone fruit,...",Quady Moscato United States Madera Quady Elect...,"['quady', 'moscato', 'united', 'states', 'made..."
22044,Indisciplinato,27.99,USD,750,3.442514,0.0,1.623086,3.174029,9.0,1.0,...,Marche,Italy,Tenuta San Marcello,//images.vivino.com/labels/kJVZn2JsRiiUc7Xfz7x...,"SEP Anfora verdicchio, not satisfied, for the ...","['dried mango', 'orange']","['dried mango', 'orange']","dried mango, orange",Tenuta San Marcello Indisciplinato Italy March...,"['tenuta', 'san', 'marcello', 'indisciplinato'..."
70,The Border Crossing,22.99,USD,750,3.083111,0.0,2.035137,3.259503,14.0,77.0,...,McLaren Vale,Australia,Tait,//images.vivino.com/thumbs/rBa8UZQNRZWShFncR8J...,"SEP An awesome Shiraz, menthol and red fruit o...","['cherry', 'menthol', 'oaky', 'red fruit', 'bl...","['cherry', 'menthol', 'oaky', 'red fruit', 'bl...","cherry, menthol, oaky, red fruit, blackberries...",Tait The Border Crossing Australia McLaren Val...,"['tait', 'border', 'crossing', 'australia', 'm..."
8219,Cuvée des Templiers Vacqueyras,29.99,USD,750,3.246583,0.0,1.662057,3.179357,66.0,75.0,...,Vacqueyras,France,Le Clos des Cazaux,//images.vivino.com/thumbs/nU82WQHZRuOd3BdS7wg...,"SEP Spicy, easy drinking, worked wonderfully w...","['game', 'smoke', 'earthy']","['game', 'smoke', 'earthy']","game, smoke, earthy",Le Clos des Cazaux Cuvée des Templiers Vacquey...,"['le', 'clos', 'des', 'cazaux', 'cuvée', 'des'..."
13382,Cabernet Sauvignon,19.99,USD,750,2.961994,0.0,1.642386,3.407533,30.0,25.0,...,Mendocino County,United States,The Seventy Five Wine Company (Est. 75),//images.vivino.com/thumbs/VRY15rozR0infWImqak...,SEP Very good value for $20 at grocery store. ...,[],['no_flavor'],no_flavor,The Seventy Five Wine Company (Est. 75) Cabern...,"['seventy', 'five', 'wine', 'company', 'est', ..."
19204,Homok,24.99,USD,750,4.814021,0.0,2.180232,2.285021,4.0,69.0,...,Burgenland,Austria,Alexander Koppitsch,//images.vivino.com/thumbs/rQId99haQk6uikaCWy_...,"SEP Cloudy yellow colour, very Refreshing\nSme...","['lemon', 'minerals', 'pineapple', 'yeast']","['lemon', 'minerals', 'pineapple', 'yeast']","lemon, minerals, pineapple, yeast",Alexander Koppitsch Homok Austria Burgenland A...,"['alexander', 'koppitsch', 'homok', 'austria',..."
7397,Cabernet Sauvignon,21.50,USD,750,3.990714,0.0,1.723643,3.930268,7.0,72.0,...,Stellenbosch,South Africa,Big Flower,//images.vivino.com/thumbs/wV36ku8vS3OTId0Nqnh...,"SEP Great Cabernet. Very tannic with bramble, ...","['tobacco', 'blackberry', 'oaky', 'plum', 'smo...","['tobacco', 'blackberry', 'oaky', 'plum', 'smo...","tobacco, blackberry, oaky, plum, smoke",Big Flower Cabernet Sauvignon South Africa Ste...,"['big', 'flower', 'cabernet', 'sauvignon', 'so..."
14332,Morgon Côte du Py,20.99,USD,750,4.568611,0.0,1.064815,1.709259,2.0,9.0,...,Morgon,France,Jean Arthaud,//images.vivino.com/thumbs/iLz--Io-RTWH13JnbMQ...,"SEP Smooth, light, so satisfying SEP Nose of c...","['balsamic', 'boysenberry', 'cherry', 'jam', '...","['balsamic', 'boysenberry', 'cherry', 'jam', '...","balsamic, boysenberry, cherry, jam, stone, bal...",Jean Arthaud Morgon Côte du Py France Morgon J...,"['jean', 'arthaud', 'morgon', 'côte', 'du', 'p..."
13431,Lirac,29.40,USD,750,3.523421,0.0,1.770852,3.215470,43.0,82.0,...,Lirac,France,Pierre Usseglio,//images.vivino.com/thumbs/LkrsEMVzQuSJvvZWyM1...,"SEP Best wine ever SEP 3,75 - Pierre Usseglio...","['dark fruit', 'licorice', 'pepper', 'clov

In [5]:
combined_text_data = wine_data['combined_text']

In [6]:
combined_text_data[0]

'Seven Daughters Moscato Italy Veneto Seven Daughters Moscato U.V. SEP Wow I was incredibly taken back by this moscato. Light yet has notes of white peach & lychee with the mid pallet some  minerals truly lovely and a perfect sweet balance not too sweet not to light.  SEP A very pale yellow.  Apple and pear.  No notable acidity. Slightly sweet but by no means cloying. A pleasant sipper   SEP Fruity and perfectly sweet lychee, minerals, white peach, apple, pear'

In [7]:
cleaned_texts = [text.replace('\n', '') for text in combined_text_data]

In [8]:
from sklearn.model_selection import train_test_split

train_texts, test_val_texts = train_test_split(cleaned_texts, test_size=0.2, random_state=42)
test_texts, val_texts = train_test_split(test_val_texts, test_size=0.5, random_state=42)

print("Liczba przykładów w zbiorze treningowym:", len(train_texts))
print("Liczba przykładów w zbiorze testowym:", len(test_texts))
print("Liczba przykładów w zbiorze walidacyjnym:", len(val_texts))

Liczba przykładów w zbiorze treningowym: 21460
Liczba przykładów w zbiorze testowym: 2682
Liczba przykładów w zbiorze walidacyjnym: 2683


In [9]:
train_texts[11]

'Dolin Vermouth France Savoie Dolin Vermouth de Chambéry Blanc N.V. SEP Floral and clean on the nose.  Caramel and honey on the palette. It would be good on the rocks SEP Perfect aperitif vermouth with a great balance of sweetness to bitterness. Pear and elderflower on the nose, with light tree fruit and lemon on the palate. This would be fantastic in a cocktail.  SEP Great herbal nose with slight sweetness. Great balance between the sweetness and the bitterness that follow. This has nice apple peel qualities as well as a little bit of pear. Very very nice.  caramel, honey, apple, pear'

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Ustawienie tokena paddingu na token końca sekwencji


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
tokenizer.save_pretrained("/content/drive/MyDrive/datasets/wines_processed/model/gpt2-finetuned/checkpoint-20000")

('/content/drive/MyDrive/datasets/wines_processed/model/gpt2-finetuned/checkpoint-20000/tokenizer_config.json',
 '/content/drive/MyDrive/datasets/wines_processed/model/gpt2-finetuned/checkpoint-20000/special_tokens_map.json',
 '/content/drive/MyDrive/datasets/wines_processed/model/gpt2-finetuned/checkpoint-20000/vocab.json',
 '/content/drive/MyDrive/datasets/wines_processed/model/gpt2-finetuned/checkpoint-20000/merges.txt',
 '/content/drive/MyDrive/datasets/wines_processed/model/gpt2-finetuned/checkpoint-20000/added_tokens.json')

In [12]:
def count_tokens(text):
    # Tokenizacja tekstu
    tokens = tokenizer(text)['input_ids']
    # Zliczenie tokenów
    return len(tokens)

In [13]:
# Obliczenie liczby tokenów dla każdej sekwencji tekstu
wine_data['num_tokens'] = wine_data['combined_text'].apply(count_tokens)

# Obliczenie średniej liczby tokenów
average_tokens = wine_data['num_tokens'].mean()

# Obliczenie maksymalnej liczby tokenów
max_tokens = wine_data['num_tokens'].max()

print("Średnia liczba tokenów: ", average_tokens)
print("Maksymalna liczba tokenów: ", max_tokens)

Średnia liczba tokenów:  150.10057781919852
Maksymalna liczba tokenów:  458


In [14]:

train_path = '/content/drive/MyDrive/datasets/wines_processed/train_data.txt'
test_path = '/content/drive/MyDrive/datasets/wines_processed/test_data.txt'
val_path = '/content/drive/MyDrive/datasets/wines_processed/val_data.txt'


with open(train_path, 'w') as f:
    for text in train_texts:
        f.write(text + '\n')


with open(test_path, 'w') as f:
    for text in test_texts:
        f.write(text + '\n')


with open(val_path, 'w') as f:
    for text in val_texts:
        f.write(text + '\n')

In [5]:
model_path = "/content/drive/MyDrive/datasets/wines_processed/model/gpt2-finetuned/checkpoint-20000"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

In [3]:
from datasets import load_dataset
train_dataset = load_dataset('text', data_files={"/content/drive/MyDrive/datasets/wines_processed/train_data.txt"})

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
train_dataset = train_dataset.map(lambda examples: tokenizer(examples["text"], return_tensors="np"), batched=True)

Map:   0%|          | 0/21467 [00:00<?, ? examples/s]

In [7]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 21467
    })
})

In [8]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/datasets/wines_processed/model/gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=6,
    save_steps=10_000,
    save_total_limit=2,
)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=train_dataset['train'],
)

In [13]:
trainer.train()

Step,Training Loss
500,1.798100
1000,0.669700
1500,0.436800
2000,0.379700
2500,0.306800
3000,0.284600
3500,0.303500
4000,0.263100
4500,0.259200
5000,0.243500


Checkpoint destination directory /content/drive/MyDrive/datasets/wines_processed/model/gpt2-finetuned/checkpoint-10000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=10734, training_loss=0.3571321033600746, metrics={'train_runtime': 1444.6299, 'train_samples_per_second': 44.58, 'train_steps_per_second': 7.43, 'total_flos': 7513297748352000.0, 'train_loss': 0.3571321033600746, 'epoch': 3.0})